In [8]:
from tensorflow.keras.models import load_model
import cv2
from PIL import Image,ImageEnhance
import numpy as np
from random import choice

In [9]:
class_map = {
    0: "rock",
    1: "paper",
    2: "scissors",
    3: "lizard",
    4: "spock",
    5: "nothing"
}

In [10]:
# Find class name by index
def class_name(index):
    return class_map[index]

In [11]:
# Game logic
def who_wins(move1, move2, player1, player2):
    if move1 == move2:
        return "Tie"

    if move1 == "rock":
        if move2 == "paper":
            return player2
        if move2 == "scissors":
            return player1
        if move2 == "lizard":
            return player1
        if move2 == "spock":
            return player2

    if move1 == "paper":
        if move2 == "rock":
            return player1
        if move2 == "scissors":
            return player2
        if move2 == "lizard":
            return player2
        if move2 == "spock":
            return player1

    if move1 == "scissors":
        if move2 == "rock":
            return player2
        if move2 == "paper":
            return player1
        if move2 == "lizard":
            return player1
        if move2 == "spock":
            return player2
        
    if move1 == "lizard":
        if move2 == "rock":
            return player2
        if move2 == "paper":
            return player1
        if move2 == "scissors":
            return player2
        if move2 == "spock":
            return player1

    if move1 == "spock":
        if move2 == "rock":
            return player1
        if move2 == "paper":
            return player2
        if move2 == "scissors":
            return player1
        if move2 == "lizard":
            return player2

In [12]:
model = load_model("models/best_model.hdf5")

In [ ]:
# Open camera (0 -> default camera)
captured_image = cv2.VideoCapture(0)

prev_move = None

while True:
    # Grab, decode and return next video frame
    retval, frame = captured_image.read() 
    if not retval:
        continue
    
    # Reverse the mirror effect
    frame = cv2.flip(frame, 1)
    
    # Get screen width and height
    window_height = frame.shape[0]
    window_width = frame.shape[1]
    
    # Rectangle for user to play
    # Def: cv2.rectangle(image, start_point, end_point, color, thickness), we set color in RGB format
    cv2.rectangle(frame, (0, 0), (int(window_width * .47), int(window_height * .65)), (255, 255, 255), 2) 
    # Rectangle for computer to play
    cv2.rectangle(frame, (int(window_width * .53), 0), (window_width, int(window_height * .65)), (255, 255, 255), 2)
    
    # Set window properties
    window_name = "Rock Paper Scissors Lizard Spock"
    cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

    # Extract the region of image within the user rectangle
    roi = frame[0:int(window_height * .60), 0:int(window_width * .40)]
    
    # Sharpen image
    im_pil = Image.fromarray(roi)
    enhancer = ImageEnhance.Sharpness(im_pil)
    im_pil = enhancer.enhance(2)
    # Reversing the operation:
    roi = np.asarray(im_pil)
    # Sharpen image
    im_pil = Image.fromarray(roi)
    enhancer = ImageEnhance.Sharpness(im_pil)
    im_pil = enhancer.enhance(2)
    # Reversing the operation:
    roi = np.asarray(im_pil)
    
    # Additional preparation of data
    img = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (300, 300))
    img = img.reshape(300, 300, 1)
    img = np.array([img]).astype('float32')
    img /= 255
    
    # Predict the move made
    pred = model.predict(img)
    move_class = np.argmax(pred[0])
    user_move_name = class_name(move_class)
    
    # Predict the winner (human vs computer)
    if prev_move != user_move_name:
        if user_move_name != "nothing":
            computer_move_name = choice(['rock', 'paper', 'scissors', 'lizard', 'spock'])
            winner = who_wins(user_move_name, computer_move_name, "User", "computer")
        else:
            computer_move_name = "nothing"
            winner = "Waiting..."
    prev_move = user_move_name

    # Display information (moves made + winner)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, "Your Move: " + user_move_name,
                (int(window_width * .05), int(window_height * .70)),
                font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (int(window_width * .65), int(window_height * .70)),
                font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Winner: " + winner,
                (int(window_width * .35), int(window_height * .90)),
                font, 0.75, (0, 0, 255), 4, cv2.LINE_AA)

    # Display computer move
    if computer_move_name != "nothing":
        icon = cv2.imread(
            "images/{}.png".format(computer_move_name))
        icon = cv2.resize(icon, (int(window_width-int(window_width * .53)), int(window_height * .65)))

        frame[0:int(window_height * .65), int(window_width * .53):window_width] = icon

    cv2.imshow(window_name, frame)

    # Set exit key
    k = cv2.waitKey(10)
    if k == ord('q'):
        break

# Release and destroy
captured_image.release()
cv2.destroyAllWindows()